This code calculates the input tension through a Gaussian approximation the way Marco suggested. The idea is the following: 

- Read the DES mean and covariance for the shifted chains
- Get a combined covariance by adding the DES and Planck covariance
- Get a difference mean as mean_DES - mean_Planck
- Get the p-value and number of sigma from a chi-squared distribution

I try using the parameter combinations $\Omega_m - \sigma_8$ and $\Omega_m - A_s$. I get **very different results for each approximation**. 

The conclusion from that is that if we pick our dimensions, the results depend on the parameterization chosen. **We should be using the full 5 dimensional posterior!!** Otherwise we are losing information and biasing our results

In [10]:
# Imports
import numpy as np
import pandas as pd
from stats import *
from scipy.stats import chi2

In [2]:
def reduce_mean(means_full, param = 'sigma8'):
    '''Convert the mean in all parameters in a 2D array containing the mean
    in Omega_m - sigma_8 or Omega_m - A_s
    '''
    if param == 'sigma8':
        index = -5
    elif param == 'As':
        index = 4
    
    mean_red = means_full[[0, index]]
    
    if param == 'As':
        mean_red[1]*=1e9
    return mean_red

def reduce_cov(cov_full, param = 'sigma8'):    
    '''Convert the covariance in all parameters in a 2D array containing the 
    covariance in Omega_m - sigma_8 or Omega_m - A_s
    '''
    if param == 'sigma8':
        index = -3
    elif param == 'As':
        index = 4
        
    cov_red = np.empty([2,2])
    cov_red[0,0] = cov_full[0,0]
    cov_red[1,0] = cov_full[index,0]
    cov_red[0,1] = cov_full[0,index]
    cov_red[1,1] = cov_full[index,index]
    
    if param == 'As':
        cov_red[1]*=1e9
        cov_red[:,1]*=1e9        
    return cov_red
    

In [3]:
# Change this to your path to the chains
PATH_TO_CHAINS = '../chains/'

In [4]:
# Read Planck data
mean_planck_full = np.loadtxt(PATH_TO_CHAINS+'chains_poly/0.01_tolerance/planck_poly0.01/means.txt', usecols = 1)
c_planck_full = np.loadtxt(PATH_TO_CHAINS+'chains_poly/0.01_tolerance/planck_poly0.01/covmat.txt') 

In [5]:
#Assign names
names = ['d_05sigD_s8', 'd_05sigU_om',
        'd_1sigD_s8', 'd_1sigU_om',
        'd_15sigD_s8', 'd_15sigU_om',
        'd_2sigD_s8', 'd_2sigU_om',
        'd_3sigD_s8', 'd_3sigU_om']

In [26]:
print('USING SIGMA 8')

# Reduce Planck data
mean_planck = reduce_mean(mean_planck_full, param = 'sigma8')
c_planck = reduce_cov(c_planck_full, param = 'sigma8')
nsigma_s8 = []

for name in names:
    
    # Read DES data
    mean_des_full = np.loadtxt(PATH_TO_CHAINS+'chains_poly/0.01_tolerance/'+name+'/means.txt', usecols = 1)
    c_des_full = np.loadtxt(PATH_TO_CHAINS+'chains_poly/0.01_tolerance/'+name+'/covmat.txt') 

    # Reduce DES data
    mean_des = reduce_mean(mean_des_full, param = 'sigma8')
    means = mean_des - mean_planck

    c_des = reduce_cov(c_des_full, param = 'sigma8')
    cov = c_des+c_planck
    
    # Calculate the number of sigma
    chi2_norm = means.dot(np.linalg.inv(cov)).dot(means)
    pval_norm = 1-chi2.cdf(chi2_norm, 2)
    nsigma = get_nsigma(pval_norm)
    nsigma_s8.append(round(nsigma, 4))
    
    # Print results
    print(name, round(nsigma, 2), 'sigma.')


USING SIGMA 8
d_05sigD_s8 0.18 sigma.
d_05sigU_om 0.96 sigma.
d_1sigD_s8 0.89 sigma.
d_1sigU_om 1.91 sigma.
d_15sigD_s8 1.91 sigma.
d_15sigU_om 2.93 sigma.
d_2sigD_s8 2.75 sigma.
d_2sigU_om 3.64 sigma.
d_3sigD_s8 4.58 sigma.
d_3sigU_om 5.44 sigma.


In [27]:
print('USING As')

# Reduce Planck data
mean_planck = reduce_mean(mean_planck_full, param = 'As')
c_planck = reduce_cov(c_planck_full, param = 'As')
nsigma_as = []

for name in names:

    # Read DES data
    mean_des_full = np.loadtxt(PATH_TO_CHAINS+'chains_poly/0.01_tolerance/'+name+'/means.txt', usecols = 1)
    c_des_full = np.loadtxt(PATH_TO_CHAINS+'chains_poly/0.01_tolerance/'+name+'/covmat.txt') 

    # Reduce DES data
    mean_des = reduce_mean(mean_des_full, param = 'As')
    means = mean_des - mean_planck
    
    c_des = reduce_cov(c_des_full, param = 'As')
    cov = c_des+c_planck
    
    # Calculate the number of sigma
    chi2_norm = means.dot(np.linalg.inv(cov)).dot(means)
    pval_norm = 1-chi2.cdf(chi2_norm, 2)
    nsigma = get_nsigma(pval_norm)
    nsigma_as.append(round(nsigma, 4))
    
    # Print results
    print(name, round(nsigma, 2), 'sigma.')


USING As
d_05sigD_s8 0.07 sigma.
d_05sigU_om 0.42 sigma.
d_1sigD_s8 0.09 sigma.
d_1sigU_om 0.87 sigma.
d_15sigD_s8 0.19 sigma.
d_15sigU_om 1.32 sigma.
d_2sigD_s8 0.45 sigma.
d_2sigU_om 1.98 sigma.
d_3sigD_s8 0.92 sigma.
d_3sigU_om 3.12 sigma.


In [44]:
columnames = ['$0.5 \sigma$ Down $\sigma_8$', '$0.5 \sigma$ Up $\Omega_m$',
              '$1 \sigma$ Down $\sigma_8$', '$1 \sigma$ Up $\Omega_m$',
              '$1.5 \sigma$ Down $\sigma_8$', '$1.5 \sigma$ Up $\Omega_m$',
              '$2 \sigma$ Down $\sigma_8$', '$2 \sigma$ Up $\Omega_m$',
              '$3 \sigma$ Down $\sigma_8$', '$3 \sigma$ Up $\Omega_m$',
             ]
data = zip(nsigma_s8, nsigma_as)
df = pd.DataFrame(data, columns = ['$\Omega_m - \sigma_8$', '$\Omega_m - A_s$'])
df.index = columnames
pd.set_option('display.width', 20000)
df

,$\Omega_m - \sigma_8$,$\Omega_m - A_s$
$0.5 \sigma$ Down $\sigma_8$,0.1842,0.0694
$0.5 \sigma$ Up $\Omega_m$,0.9601,0.4171
$1 \sigma$ Down $\sigma_8$,0.8891,0.0929
$1 \sigma$ Up $\Omega_m$,1.9098,0.8682
$1.5 \sigma$ Down $\sigma_8$,1.9122,0.1928
$1.5 \sigma$ Up $\Omega_m$,2.9336,1.3166
$2 \sigma$ Down $\sigma_8$,2.7500,0.4538
$2 \sigma$ Up $\Omega_m$,3.6447,1.9842
$3 \sigma$ Down $\sigma_8$,4.5788,0.9199
$3 \sigma$ Up $\Omega_m$,5.4394,3.1163
